# Análise das Stories

Notebook para o auxílio da análise das stories da Taís.


### Configurando jupyter

In [1]:
# Checking versions
import rasa_nlu
import rasa_core
from IPython.display import IFrame

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


rasa_nlu: 0.14.1 rasa_core: 0.13.1


# Análise e Avaliação das Stories

### Visualizaçõa do Fluxo de Conversa

O comando abaixo monta um grafo com a relação das `intents` e `utters` do chatbot, ou seja, você vai conseguir visualizar o fluxo de conversa do seu chatbot

Esta visualização é importante para verificar possíveis problemas na estrutura do seu chatbot e se ele realmente chega em determinadas "**falas**" da conversa.

In [2]:
!python -m rasa_core.visualize -d ../../bot/domain.yml -s ../../bot/data/stories/ -o chat_graph.html

/usr/local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2019-05-03 15:00:41 INFO     __main__  - Starting to visualize stories...
Processed Story Blocks: 100%|███| 34/34 [00:00<00:00, 7209.62it/s, # trackers=1]
2019-05-03 15:00:41 INFO     __main__  - Finished graph creation. Saved into file:///work/notebooks/stories/chat_graph.html


O Rasa gerou uma página `html` com o grafo de conversa, para facilitar, você pode visualizar o grafo no arquivo `chat_graph` aqui na próxima celula.

Use o mouse para dar zoom e arrastar sobre o conteúdo do grafo.

In [3]:
IFrame(src='./chat_graph.html', width=900, height=700)